In [3]:
# DON'T RUN THIS BLOCK AGAIN

import configparser
import json
from datetime import datetime
from telethon import TelegramClient
from telethon.errors import SessionPasswordNeededError

# Reading Configs
config = configparser.ConfigParser()
config.read("config.ini")

# Setting configuration values
api_id = config['Telegram']['api_id']
api_hash = config['Telegram']['api_hash']

api_hash = str(api_hash)

phone = config['Telegram']['phone']
username = config['Telegram']['username']

# Create the client and connect
client = TelegramClient(username, api_id, api_hash)
await client.start()
print("Client Created")

Client Created


In [4]:
# DON'T RUN THIS BLOCK AGAIN

# Ensure you're authorized
if not client.is_user_authorized():
    client.send_code_request(phone)
    try:
        client.sign_in(phone, input('Enter the code: '))
    except SessionPasswordNeededError:
        client.sign_in(password=input('Password: '))

from telethon.tl.types import PeerChannel

# user_input_channel = input("enter entity(telegram URL or entity id):")
user_input_channel = '1001546061018'

if user_input_channel.isdigit():
    entity = PeerChannel(-int(user_input_channel))
else:
    entity = user_input_channel

await client.connect()
my_channel = await client.get_entity(entity)

/var/folders/ym/67x0srsx2ln9vy4qtlvtdv500000gn/T/ipykernel_18315/2990546721.py:4: RuntimeWarning: coroutine 'UserMethods.is_user_authorized' was never awaited
  if not client.is_user_authorized():


In [5]:
# Getting Channel messages
from telethon.tl.functions.messages import GetHistoryRequest
offset_id = 0
limit = 100
all_messages = []
total_messages = 0
total_count_limit = 300

while True:
#     print("Current Offset ID is:", offset_id, "; Total Messages:", total_messages)
    history = await client(GetHistoryRequest(
        peer=my_channel,
        offset_id=offset_id,
        offset_date=datetime.now(),
        add_offset=0,
        limit=limit,
        max_id=0,
        min_id=0,
        hash=0
    ))
    if not history.messages:
        break
    messages = history.messages
    for message in messages:
        all_messages.append(message.to_dict())
    offset_id = messages[len(messages) - 1].id
    total_messages = len(all_messages)
    if total_count_limit != 0 and total_messages >= total_count_limit:
        break
print("Total Messages:", total_messages)

Total Messages: 300


In [8]:
a_long_call = 0
a_long_put = 0
a_short_call = 0
a_short_put = 0
b_long_call = 0
b_long_put = 0
b_short_call = 0
b_short_put = 0
iteration = 1
due_a_object = 0
due_b_object = 0

date_now = datetime(2022, 7, 11)
date = date_now.strftime('%Y-%m-%d')


for item in all_messages:
    content = item['message']
    
    due_index = content.index('到期日:') + 5
    if iteration == 1:
        due_a = content[due_index:due_index + 10]
    if iteration >= 1 and due_a < content[due_index:due_index + 10]:
        due_b = content[due_index:due_index + 10]
        due_b_object = datetime.fromisoformat(due_b)
    iteration += 1
    
    buy_index = content.index('成交日期') + 5
    buy_date = content[buy_index:buy_index + 10]
    datetime_object = datetime.fromisoformat(buy_date)
    
    due_date = content[due_index:due_index + 10]
    due_date = datetime.fromisoformat(due_date)
    
    
    if datetime_object >= date_now:
        if due_b_object != due_date:
            if 'LONG CALL' in content:
                a_long_call += 1
            if 'LONG PUT' in content:
                a_long_put += 1
            if 'SHORT CALL' in content:
                a_short_call += 1
            if 'SHORT PUT' in content:
                a_short_put += 1
                
        if due_b_object == due_date:
            if 'LONG CALL' in content:
                b_long_call += 1
            if 'LONG PUT' in content:
                b_long_put += 1
            if 'SHORT CALL' in content:
                b_short_call += 1
            if 'SHORT PUT' in content:
                b_short_put += 1
                
                
print('Option alert of', date,':')
print('\033[1mDue',due_a,':\033[0m')
print('Number of LONG CALL:', a_long_call)
print('Number of LONG PUT:', a_long_put)
print('Number of SHORT CALL:', a_short_call)
print('Number of SHORT PUT:', a_short_put)
print('Net CALL:', a_long_call - a_short_call)
print('Net PUT:', a_long_put - a_short_put)
print('Total number of alerts:', a_long_call + a_long_put + a_short_call + a_short_put)
print('\033[1mDue',due_b,':\033[0m')
print('Number of LONG CALL:', b_long_call)
print('Number of LONG PUT:', b_long_put)
print('Number of SHORT CALL:', b_short_call)
print('Number of SHORT PUT:', b_short_put)
print('Net CALL:', b_long_call - b_short_call)
print('Net PUT:', b_long_put - b_short_put)
print('Total number of alerts:', b_long_call + b_long_put + b_short_call + b_short_put)

Option alert of 2022-07-11 :
Due 2022-07-15 :
Number of LONG CALL: 13
Number of LONG PUT: 27
Number of SHORT CALL: 11
Number of SHORT PUT: 15
Net CALL: 2
Net PUT: 12
Total number of alerts: 66
Due 2022-07-22 :
Number of LONG CALL: 2
Number of LONG PUT: 3
Number of SHORT CALL: 2
Number of SHORT PUT: 0
Net CALL: 0
Net PUT: 3
Total number of alerts: 7


In [9]:
print(content)

#BYND
SHORT PUT 
行權價:$45.00
正股股價:$31.37

到期日: 2022-07-15
張數:138
未平倉張數:1103
成交額USD:$202170
成交價:$14.65
成交:AT_BID
成交日期:2022-07-07 | 成交時間: 14:15:56 

詳情:Beyond Meat Option Alert: Jul 15 $45 Puts at the Bid: 138 @ $14.65 vs 1103 OI; Earnings 8/4 After Close [est] Ref=$31.37
